### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [1]:
%%capture
!pip install transformers

In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/쿠빅/25-여름 NLP 세션/bbc-text.csv') # bbc-text.csv 파일 경로

In [5]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
print(len(df))

2225


In [7]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [9]:
class Dataset(torch.utils.data.Dataset):
    # 초기화 함수: 데이터프레임을 받아서 라벨과 텍스트를 BERT 입력 형식으로 변환
    def __init__(self, df):
        # 카테고리 문자열을 숫자 라벨로 변환 (예: 'sport' → 0)
        self.labels = [labels[label] for label in df['category']]

        # 각 텍스트를 BERT 토크나이저로 인코딩하여 input_ids, attention_mask를 생성
        self.texts = [
            tokenizer(
                text,
                padding='max_length',       # 길이가 짧은 문장은 max_length까지 패딩
                max_length=512,             # BERT 입력 최대 길이
                truncation=True,            # 길이가 너무 길면 자름
                return_tensors="pt"         # PyTorch 텐서로 반환 (dict 형태)
            )
            for text in df['text']
        ]

    # 라벨 리스트 전체를 반환하는 함수 (사용 안 할 수도 있음)
    def classes(self):
        return self.labels

    # 전체 샘플 개수 반환 (len(dataset) 형태로 사용됨)
    def __len__(self):
        return len(self.labels)

    # 특정 인덱스의 라벨 반환
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])  # numpy array 형태로 반환

    # 특정 인덱스의 텍스트 토큰 정보 반환 (tokenizer 결과)
    def get_batch_texts(self, idx):
        return self.texts[idx]  # {'input_ids': tensor, 'attention_mask': tensor}

    # DataLoader가 데이터를 불러올 때 호출되는 함수
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)     # 인코딩된 텍스트
        batch_y = self.get_batch_labels(idx)        # 정답 라벨
        return batch_texts, batch_y                 # 둘 다 반환 (model에 직접 넣을 수 있음)


## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [10]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()

        # 사전 학습된 BERT 모델 로딩 (대소문자 구분하는 cased 버전)
        self.bert = BertModel.from_pretrained('bert-base-cased')

        # Dropout은 과적합 방지를 위해 사용
        self.dropout = nn.Dropout(dropout)

        # BERT의 출력 차원은 768 → 분류 클래스 수만큼 출력 (여기선 5개 카테고리)
        self.linear = nn.Linear(768, 5)

        # 활성화 함수 (ReLU): 비선형성 부여
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        # BERT에 input_ids와 attention_mask 입력
        # return_dict=False → 튜플 형태 반환
        _, pooled_output = self.bert(
            input_ids=input_id,
            attention_mask=mask,
            return_dict=False
        )

        # pooled_output: [CLS] 토큰의 출력 벡터 (전체 문장을 대표함)
        dropout_output = self.dropout(pooled_output)      # Dropout 적용
        linear_output = self.linear(dropout_output)       # 선형 변환 (768 → 5)
        final_layer = self.relu(linear_output)            # ReLU 적용 후 반환

        return final_layer                                # 예측 결과 (logits)


In [11]:
def train(model, train_data, val_data, learning_rate, epochs):

    # 훈련용/검증용 데이터셋 생성 (Dataset 클래스 활용)
    train, val = Dataset(train_data), Dataset(val_data)

    # PyTorch DataLoader를 통해 배치 단위로 로딩 (배치 사이즈 2)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # GPU 사용 가능 여부 확인 및 설정
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 분류 문제이므로 CrossEntropyLoss 사용 (softmax + log loss 조합)
    criterion = nn.CrossEntropyLoss()

    # BERT 모델의 모든 파라미터를 Adam 옵티마이저로 학습
    optimizer = Adam(model.parameters(), lr=learning_rate)

    # GPU 사용 설정 (모델과 손실 함수)
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # 에폭 루프 시작
    for epoch_num in range(epochs):
        total_acc_train = 0       # 전체 정답 개수
        total_loss_train = 0      # 전체 loss 합산

        # 학습 데이터 루프
        for train_input, train_label in tqdm(train_dataloader):

            # 정답 라벨, attention mask, input_ids를 GPU로 이동
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            # 모델에 입력 → 예측 결과 (logits)
            output = model(input_id, mask)

            # 손실 계산 (정답 라벨과 예측 결과 비교)
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            # 정확도 계산: 예측한 클래스와 실제 정답 비교
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            # 역전파 수행
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        # 검증 루프 시작 (gradient 계산 X)
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                # 검증용 forward pass
                output = model(input_id, mask)

                # 손실 및 정확도 계산
                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        # 에폭 결과 출력
        print(
            f'Epochs: {epoch_num + 1} | '
            f'Train Loss: {total_loss_train / len(train_data): .3f} | '
            f'Train Accuracy: {total_acc_train / len(train_data): .3f} | '
            f'Val Loss: {total_loss_val / len(val_data): .3f} | '
            f'Val Accuracy: {total_acc_val / len(val_data): .3f}'
        )


In [12]:
def evaluate(model, test_data):
    # 테스트 데이터셋을 커스텀 Dataset 클래스를 이용해 변환
    test = Dataset(test_data)

    # DataLoader 생성 (배치 크기 2)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    # GPU 사용 여부 확인 및 설정
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()  # 모델을 GPU로 이동

    total_acc_test = 0  # 전체 정답 개수 누적 변수

    # 평가 시에는 gradient 계산 불필요 → 성능/속도 ↑
    with torch.no_grad():
        for test_input, test_label in test_dataloader:

            # 입력값과 정답 라벨을 device(GPU or CPU)로 이동
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            # 모델 예측
            output = model(input_id, mask)

            # 예측값과 정답 비교하여 맞춘 개수 계산
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    # 전체 정확도 계산 및 출력
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')


In [13]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

  0%|          | 0/890 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 890/890 [03:17<00:00,  4.51it/s]


Epochs: 1 | Train Loss:  0.674 | Train Accuracy:  0.471 | Val Loss:  0.510 | Val Accuracy:  0.725


100%|██████████| 890/890 [03:19<00:00,  4.47it/s]


Epochs: 2 | Train Loss:  0.351 | Train Accuracy:  0.877 | Val Loss:  0.196 | Val Accuracy:  0.982


In [15]:
evaluate(model, df_test)

Test Accuracy:  0.978


optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗